In [3]:
import numpy as np 
import sklearn.metrics as mt
from keras.models import load_model
import pickle
import sklearn as sk
import sklearn.preprocessing

# Setup Testing Data

In [4]:
with open('NonProg_final_test', 'rb') as rf:
    mfcc_nonProg = pickle.load(rf)

for song in range(len(mfcc_nonProg)):
    mfcc_nonProg[song] = sk.preprocessing.scale(mfcc_nonProg[song], axis = 1)

with open('Prog_final_test', 'rb') as rf:
    mfcc_Prog = pickle.load(rf)

for song in range(len(mfcc_Prog)):
    mfcc_Prog[song] = sk.preprocessing.scale(mfcc_Prog[song], axis = 1)
    
with open('djent_mfcc', 'rb') as rf:
    mfcc_djent = pickle.load(rf)

for song in range(len(mfcc_djent)):
    mfcc_djent[song] = sk.preprocessing.scale(mfcc_djent[song], axis = 1)


mfcc_test = mfcc_nonProg + mfcc_Prog
#mfcc_test = mfcc_djent

# First LSTM Model

In [9]:
misclassified = 0 

model = load_model('mfcc_ts_20_3Layers.h5')
label = np.zeros(len(mfcc_test))
print(label.shape)

for song_num, item in enumerate(mfcc_test): 
    #Split the song into segments of 100 frames
    total_frames = np.shape(item)[1]
    num_segs = total_frames//100
    test_data = np.zeros([num_segs, 100 , 20])
    mark = 0
    for i in range (num_segs):
        temp = item[:,mark:mark+100].T
        test_data[i,:,:] = temp
        mark += 100
    #Predict the class of each segment
    y = model.predict_classes(test_data)
    votes = sum(y)
    #print(str(song_num) + " " + str(votes) + " " + str(num_segs))
    #Determine class by looking at the votes for each segment
    if votes > num_segs//2 : 
        label[song_num] = 1

#Create the true labels for the test data 
a = np.ones(101, dtype=np.int)
b = np.zeros(101, dtype=np.int)            
true_label = np.concatenate((a,b), axis=None)

#Compare the true label to the predicted label
accuracy = mt.accuracy_score(true_label,label )
print(accuracy)
conf_matrix = mt.confusion_matrix(true_label, label)
print(conf_matrix)
print(label)
misclassified = []
#Create test data for the next LSTM model from the misclassified songs
label_nextModel = []
nextModel_data = []
for i,num in enumerate(label):
    if num != true_label[i]:
        misclassified.append(i)
        nextModel_data.append(mfcc_test[i])
        label_nextModel.append(true_label[i])
    else: 
        continue
print(misclassified)


(202,)
0 50 106
1 50 154
2 152 167
3 100 124
4 111 300
5 144 149
6 89 91
7 36 81
8 116 230
9 130 141
10 110 114
11 78 129
12 87 143
13 631 853
14 85 123
15 59 91
16 85 98
17 78 92
18 124 218
19 53 75
20 69 98
21 94 149
22 64 90
23 65 69
24 42 112
25 87 156
26 124 160
27 79 88
28 145 167
29 76 96
30 63 64
31 75 89
32 69 90
33 96 158
34 193 216
35 468 688
36 107 110
37 156 215
38 95 108
39 110 125
40 67 124
41 81 120
42 56 104
43 78 91
44 157 363
45 126 174
46 50 86
47 64 87
48 58 82
49 110 165
50 67 146
51 86 104
52 57 78
53 63 98
54 66 91
55 202 234
56 101 159
57 75 102
58 134 142
59 51 84
60 84 95
61 93 145
62 68 88
63 98 128
64 115 163
65 60 81
66 24 53
67 38 85
68 73 151
69 98 295
70 358 402
71 62 66
72 168 205
73 112 122
74 101 107
75 150 163
76 118 127
77 19 62
78 66 92
79 89 99
80 57 98
81 82 96
82 54 185
83 39 109
84 83 111
85 187 190
86 56 71
87 42 72
88 101 111
89 86 100
90 43 99
91 47 127
92 99 103
93 91 136
94 130 161
95 148 213
96 75 100
97 62 117
98 176 215
99 93 99
100 11

# Second LSTM Model

In [10]:
model1 = load_model('mfcc_ts_rockMetal_regularized.h5')
#label_comp = np.zeros(len(mfcc_test))
#for song_num, item in enumerate(mfcc_test): 
label_comp = np.zeros(len(nextModel_data))
for song_num, item in enumerate(nextModel_data): 
    #Split the song into segments of 100 frames
    total_frames = np.shape(item)[1]
    num_segs = total_frames//100
    test_data = np.zeros([num_segs, 100 , 20])
    mark = 0
    for i in range (num_segs):
        temp = item[:,mark:mark+100].T
        test_data[i,:,:] = temp
        mark += 100
    #Predict the class of each segment   
    y = model1.predict_classes(test_data)
    votes = sum(y)
    #print(str(song_num) + " " + str(votes) + " " + str(num_segs))
    if votes> num_segs//2 :
        
        label_comp[song_num] = 1
        
accuracy = mt.accuracy_score(label_nextModel,label_comp )

conf_matrix = mt.confusion_matrix(label_nextModel, label_comp)
print(conf_matrix)

print(label_nextModel)
print(label_comp)

0 42 106
1 49 154
2 12 300
3 74 81
4 60 112
5 40 363
6 51 146
7 32 53
8 23 85
9 14 151
10 23 295
11 24 62
12 89 185
13 44 109
14 67 99
15 11 127
16 128 148
17 70 103
18 63 261
19 97 176
20 83 323
21 17 128
22 45 202
23 110 286
24 299 479
25 76 275
26 38 236
27 96 145
28 115 259
29 51 107
30 241 588
31 185 259
32 91 168
33 97 155
34 19 167
35 93 501
36 39 169
37 54 157
38 55 193
39 42 129
[[16  8]
 [12  4]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]


# Accuracy and Confusion Matrix for Pipeline

In [7]:
for i,num in enumerate(label_comp):
    if num == label_nextModel[i]:
        label[misclassified[i]] = num 
    else: 
        continue
        
accuracy = mt.accuracy_score(true_label,label)
print("pipeline accuracy = "+ str(accuracy))
conf_matrix = mt.confusion_matrix(true_label, label)
print("Confusion Matrix: ")
print(conf_matrix)

pipeline accuracy = 0.900990099009901
Confusion Matrix: 
[[93  8]
 [12 89]]


In [8]:
print(misclassified[34])
print(misclassified[35])
print(misclassified[36])
print(misclassified[37])
print(misclassified[38])
print(misclassified[39])

print(len(misclassified))




184
188
192
196
197
200
40
